<a href="https://colab.research.google.com/github/vrangayyan6/GenAI/blob/main/Cerebras_Multi_agent_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using AI to Streamline Research for Content Creation
## The Problem
Content creators, researchers, and marketers face a significant challenge when developing comprehensive material on specialized topics. The traditional research process is both time-consuming and labor-intensive:

They must formulate effective search queries
Sift through numerous search results manually
Evaluate the relevance and credibility of sources
Extract and organize key information
Synthesize findings into coherent content
Repeat this process multiple times to fill knowledge gaps

This workflow can take hours or even days, delaying content production and limiting the number of topics a team can cover effectively. For small teams or individual creators without research assistants, this bottleneck severely impacts productivity.

## The AI Solution
Generative AI, as demonstrated in the code you shared, can transform this process through automated research assistance:

- Query Optimization: The AI refines user queries to match search engine algorithms for better results (as shown in the format_search method)
- Automated Information Gathering: Instead of manual searching, the AI conducts multiple search queries and aggregates the results
- Intelligent Gap Analysis: The system evaluates the completeness of research and automatically identifies missing information (via the EditorAgent)
- Iterative Research: The AI conducts multiple rounds of research until sufficient information is gathered, targeting different aspects of the topic with each iteration
- Content Synthesis: When research is complete, the AI transforms raw research into well-structured content (via the WriterAgent)

## Real-World Impact
This AI-powered research workflow reduces what might take hours into minutes. Content creators can focus on refining and adding their unique perspective to AI-generated drafts rather than spending time on initial research and organization.
The solution is particularly valuable for:

- Marketing teams needing to create content across multiple product lines
- Researchers exploring new domains quickly
- Educational content creators covering diverse topics
- Small businesses without dedicated research staff

By increasing the number of search results (as you've requested), the system becomes even more effective, gathering a wider range of perspectives and information in each research iteration.

# Multi Agentic Workflow with Cerebras, Google, LangChain and LangGraph

Got early access to Meta’s latest model, Llama 4, running at 2611 tok/s, the fastest inference speed available at [cloud.cerebras.ai](https://cloud.cerebras.ai) and trying it out in this notebook.

https://inference-docs.cerebras.ai/introduction

# Cerebras API Key
Get Cerebras API key at https://cloud.cerebras.ai/

# LangChain key
Follow steps in
https://docs.smith.langchain.com/administration/how_to_guides/organization_management/create_account_api_key

# Setting up Google Search API Credentials

Before running the main code, you need to set up your Google Search API credentials:

## Step 1: Get a Google API Key
1. Go to the [Google Cloud Console](https://console.cloud.google.com/)
2. Create a new project or select an existing one
3. Enable the "Custom Search API" for your project
4. Go to "Credentials" and create an API key

## Step 2: Create a Programmable Search Engine
1. Go to [Programmable Search Engine](https://programmablesearchengine.google.com/about/)
2. Click "Create a Programmable Search Engine"
3. Configure your search engine (you can search the entire web)
4. After creation, find your "Search engine ID" (also called CSE ID)


In [1]:
!pip install -q langchain_cerebras langchain_community langgraph langchain langchain-core langsmith langchain_experimental cerebras_cloud_sdk langchain-google-community google-search-results requests beautifulsoup4


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━

## Configure API Keys
Add API keys in Secrets (left menu)

In [2]:
import os
from google.colab import userdata

# Make sure these are set BEFORE creating any search wrappers
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLESEARCH_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
api_key = userdata.get('CEREBRAS_API_KEY')

## Enable Tracing

LangChain offers an optional tracing feature that helps in debugging complex chains and workflows.

In [3]:
# Add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Setup Data Storage

We initialize a simple in-memory list to store results from the agent workflow:

In [4]:
final_result = []

## Define Shared State

We use a TypedDict class to define and track the shared state between agents. This makes it easier to debug and enforce consistency.

In [5]:
from langgraph.graph import StateGraph
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from typing import Annotated, List, Dict, Tuple

class State(TypedDict):
    query: Annotated[list, add_messages]
    url: Annotated[list, add_messages]
    research: Annotated[list, add_messages]
    content: str
    content_ready: bool
    iteration_count: int     # Counter for iterations

## Initialize the Language Model

We use Google's Gemini model via LangChain’s wrapper.

In [6]:
from langchain_cerebras import ChatCerebras

# Initialize ChatCerebras instance for language model
llm = ChatCerebras(api_key=api_key, model="llama-4-scout-17b-16e-instruct")

## Research Agent

format_search(): Uses Gemini to transform natural queries into more search-optimized forms.

search(): Executes a search via GoogleSearchRun and stores results in state.

🔁 Iteration results are logged in final_result for each loop of query → search.

In [7]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_google_community import GoogleSearchRun
from langchain_core.messages import HumanMessage
import requests
from bs4 import BeautifulSoup
import time

class ResearchAgent:
    def format_search(self, query: str) -> str:
        prompt = (
            "You are an expert at optimizing search queries for Google. "
            "Your task is to take a given query and return an optimized version of it, making it more likely to yield relevant results. "
            "Do not include any explanations or extra text, only the optimized query.\n\n"
            "Example:\n"
            "Original: best laptop 2023 for programming\n"
            "Optimized: top laptops 2023 for coding\n\n"
            "Example:\n"
            "Original: how to train a puppy not to bite\n"
            "Optimized: puppy training tips to prevent biting\n\n"
            "Now optimize the following query:\n"
            f"Original: {query}\n"
            "Optimized:"
        )

        response = llm.invoke(prompt)
        return response.content

    def fetch_full_content(self, url: str) -> str:
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
            response = requests.get(url, headers=headers, timeout=30)
            response.raise_for_status()  # Raise an exception for bad status codes
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract text content - you might need to adjust the selectors
            text_parts = soup.find_all('p')  # Example: get all paragraph text
            full_text = "\n".join([part.get_text() for part in text_parts])
            if full_text=="":
                full_text = soup.get_text()[:5000]
            return f"{full_text[:5000]} " # Limit content length
        except requests.exceptions.RequestException as e:
            return f"URL: {url}\nError fetching content: {e}"
        except Exception as e:
            return f"URL: {url}\nError parsing content: {e}"

    def search(self, state: State):    #  -> Dict[str, List[HumanMessage]]
        google_search = GoogleSearchAPIWrapper(k=10)
        optimized_query = self.format_search(state.get('query', "")[-1].content)
        for _ in range(3):
            raw_results = google_search.results(optimized_query, 1)

            if raw_results and raw_results[0].get('link'):
                url = raw_results[0]['link']
                full_content = self.fetch_full_content(url)
                time.sleep(1) # Be respectful of website rate limits
                if full_content!="":
                    final_result.append({"subheader": f"Research Iteration", "content": [full_content], "time": time.perf_counter() - time.perf_counter()}) # Correct the time calculation
                    return {"research": full_content, "url": url, "query": optimized_query}
                else:
                    print("No content found")
            else:
                print("No url found")

        return {"query": optimized_query}


## Editor Agent
Uses Gemini to inspect accumulated research.

If results are sufficient, it sets content_ready = True.

If insufficient, it generates a new improved query and continues the loop.

🔐 Includes a hard limit of 10 iterations to prevent infinite loops.

In [8]:
class EditorAgent:
    def evaluate_research(self, state: State):
        query = '\n'.join(message.content for message in state.get("query"))
        research_content = "\n".join([message.content for message in state.get("research")])

        iteration_count = state.get("iteration_count", 1)

        if iteration_count is None:
            iteration_count = 1

        if iteration_count >= 3:
            return {"content_ready": True}

        prompt = (
            "You are an expert editor. Your task is to evaluate the research based on the query. "
            "If the information is sufficient to create a comprehensive and accurate blog post, respond with 'sufficient'. "
            "If the information is not sufficient, respond with 'insufficient' and provide a new, creative query suggestion to improve the results. "
            "If the research results appear repetitive or not diverse enough, think about a very different kind of question that could yield more varied and relevant information. "
            "Consider the depth, relevance, and completeness of the information when making your decision.\n\n"
            "Example 1:\n"
            "Used queries: What are the benefits of a Mediterranean diet?\n"
            "Research: The Mediterranean diet includes fruits, vegetables, whole grains, and healthy fats.\n"
            "Evaluation: Insufficient\n"
            "New query: Detailed health benefits of a Mediterranean diet\n\n"
            "Example 2:\n"
            "Used queries: How does solar power work?\n"
            "Research: Solar power works by converting sunlight into electricity using photovoltaic cells.\n"
            "Evaluation: Sufficient\n\n"
            "Example 3:\n"
            "Used queries: Effects of climate change on polar bears?\n"
            "Research: Climate change is reducing sea ice, affecting polar bear habitats.\n"
            "Evaluation: Insufficient\n"
            "New query: How are polar bears adapting to the loss of sea ice due to climate change?\n\n"
            "Now evaluate the following:\n"
            f"Used queries: {query}\n"
            f"Research: {research_content}\n\n"
            "Evaluation (sufficient/insufficient):\n"
            "New query (if insufficient):"
        )

        start_time = time.perf_counter()
        response = llm.invoke(prompt)
        end_time = time.perf_counter()

        evaluation = response.content.strip()

        final_result.append({"subheader": f"Editor Evaluation Iteration", "content": evaluation, "time": end_time - start_time})

        if "new query:" in evaluation.lower():
            new_query = evaluation.split("New query:", 1)[-1].strip()
            return {"query": [new_query], "iteration_count": iteration_count + 1, "evaluation": evaluation}
        else:
            return {"content_ready": True, "evaluation": evaluation}


## Writer Agent
Combines query and research into a single blog post.

Uses Gemini to generate detailed, structured content (intro, body, conclusion).

In [9]:
class WriterAgent:
    def write_blogpost(self, state: Dict) -> Dict[str, str]:
        # Handle query access more safely
        query = state.get("query", [""])[0].content if isinstance(state.get("query", [""]), list) else state.get("query", "")
        context = "\n".join([f"[{i+1}] {message.content} \n" for i, message in enumerate(state.get("research", []))])
        references = "\n".join([f"[{i+1}] {message.content} \n" for i, message in enumerate(state.get("url", []))])

        prompt = (
            "You are an expert blog post writer. Your task is to take a given query and context, and write a comprehensive, engaging, and informative short blog post about it. "
            "Make sure to include an introduction, main body with detailed information, and a conclusion. Use a friendly and accessible tone, and ensure the content is well-structured and easy to read. "
            "Apply best practices to cite the sources you used by referring to the number in the square brackets. "
            "Do not add the References section, I will add it at the bottom of the blog post."
            "Do not add anything other content not in the sources.\n\n"
            f"Query: {query}\n\n"
            f"Context:\n{context}\n\n"
            f"**References:**\n{references}"
            "Write a detailed and engaging blog post based on the above query and context."
        )

        response = llm.invoke(prompt)
        blogpost = f"{response.content}\n\n**References:**\n\n{references}"

        return {"content": blogpost}

## Define the LangGraph
🔄 This creates a loop between search_agent → editor_agent → (search or writer).

In [10]:
from langgraph.graph import END

# Initialize the StateGraph
graph = StateGraph(State)

research_agent = ResearchAgent()
editor_agent = EditorAgent()
writer_agent = WriterAgent()


graph.add_node("search_agent", research_agent.search)
graph.add_node("editor_agent", editor_agent.evaluate_research)
graph.add_node("writer_agent", writer_agent.write_blogpost)


graph.set_entry_point("search_agent")

graph.add_edge("search_agent", "editor_agent")

graph.add_conditional_edges(
    "editor_agent",
    lambda state: "accept" if state.get("content_ready") else "revise",
    {
        "accept": "writer_agent",
        "revise": "search_agent"
    }
)

graph.add_edge("writer_agent", END)

graph = graph.compile()

In [11]:
from IPython.display import Image

# Image(graph.get_graph().draw_mermaid_png())

## Main Invocation Function
This function:

Starts the LangGraph with the user’s query

Automatically loops through agents until content_ready

Returns the final blog post

In [12]:
def invoke_graph(user_prompt):
    start_time = time.perf_counter()
    result = graph.invoke({"query": user_prompt})
    end_time = time.perf_counter()
    print("\n\n")
    print(f"Total time in completing the workflow: {end_time - start_time} seconds")
    print("\n\n")
    return result["content"]

# Provide your prompt below

In [13]:
user_prompt = "Act as an expert in Financial Services, explain in detail on separately managed accounts using 5000 words or more."

# View response
You will see the Google search results in the first section, and the blogpost generated in the next.

In [14]:
from IPython.display import Markdown
result = invoke_graph(user_prompt)
Markdown(result)




Total time in completing the workflow: 9.165359447000014 seconds





**The World of Separately Managed Accounts: A Comprehensive Guide**

In the realm of financial services, investors are constantly seeking ways to diversify their portfolios and achieve their financial goals. One investment vehicle that has gained popularity in recent years is the separately managed account (SMA). In this blog post, we will delve into the world of SMAs, exploring how they work, their benefits, and what investors need to know.

**What is a Separately Managed Account?**

A separately managed account is a type of investment account that allows investors to own individual securities directly. Unlike mutual funds or exchange-traded funds (ETFs), SMAs offer a customized investment portfolio tailored to an individual's specific needs and goals [1]. In an SMA, the investor owns the underlying securities, and the investment manager has discretion to buy and sell securities on behalf of the investor.

**How Does a Separately Managed Account Work?**

Here's a step-by-step overview of how an SMA works:

1. **Account Setup**: An investor opens an SMA with a financial institution or investment manager.
2. **Investment Objective**: The investor and investment manager discuss and agree on an investment objective, such as growth, income, or capital preservation.
3. **Portfolio Construction**: The investment manager creates a customized portfolio of individual securities that aligns with the investor's objective.
4. **Portfolio Management**: The investment manager actively manages the portfolio, buying and selling securities as needed to achieve the investor's objective.
5. **Reporting and Monitoring**: The investment manager provides regular reporting and monitoring of the portfolio's performance.

**Benefits of Separately Managed Accounts**

SMAs offer several benefits to investors, including:

* **Customization**: SMAs allow investors to have a tailored investment portfolio that meets their specific needs and goals.
* **Transparency**: Investors have complete visibility into the underlying securities and can see exactly what they own.
* **Control**: Investors have more control over their investments, as they can make changes to the portfolio as needed.
* **Tax Efficiency**: SMAs can be more tax-efficient than mutual funds or ETFs, as investors only pay taxes on the capital gains realized in their account.

**Types of Separately Managed Accounts**

There are several types of SMAs available, including:

* **Equity SMAs**: Focus on stocks and equities.
* **Fixed Income SMAs**: Focus on bonds and fixed income securities.
* **Multi-Asset SMAs**: Offer a diversified portfolio of stocks, bonds, and other asset classes.

**Who is a Separately Managed Account Suitable For?**

SMAs are suitable for a wide range of investors, including:

* **High Net Worth Individuals**: SMAs offer a high level of customization and control, making them an attractive option for high net worth individuals.
* **Institutional Investors**: SMAs can be used by institutional investors, such as pension funds or endowments, to manage their investment portfolios.
* **Financial Advisors**: Financial advisors can use SMAs to offer customized investment solutions to their clients.

**Best Practices for Investing in Separately Managed Accounts**

When investing in an SMA, it's essential to:

* **Clearly Define Your Investment Objective**: Work with your investment manager to define your investment objective and risk tolerance.
* **Choose an Experienced Investment Manager**: Select an investment manager with a proven track record of success in managing SMAs.
* **Monitor and Adjust Your Portfolio**: Regularly review your portfolio and make adjustments as needed to ensure it remains aligned with your investment objective.

**Conclusion**

Separately managed accounts offer a unique and flexible investment solution for investors seeking customization and control. By understanding how SMAs work and their benefits, investors can make informed decisions about whether an SMA is right for them. Whether you're a high net worth individual or an institutional investor, SMAs can provide a tailored investment portfolio that meets your specific needs and goals.

For more information on SMAs, investors can refer to [1] and [2]. Additionally, for those interested in sustainable and impact investing, the US SIF organization provides valuable resources and education [3].

**References:**

[1] https://smartasset.com/investing/separately-managed-account 

[2] https://am.jpmorgan.com/content/dam/jpm-am-aem/americas/us/en/smas/bro-sma.pdf 

[3] https://www.ussif.org/sribasics 


In [15]:
print(result)

**The World of Separately Managed Accounts: A Comprehensive Guide**

In the realm of financial services, investors are constantly seeking ways to diversify their portfolios and achieve their financial goals. One investment vehicle that has gained popularity in recent years is the separately managed account (SMA). In this blog post, we will delve into the world of SMAs, exploring how they work, their benefits, and what investors need to know.

**What is a Separately Managed Account?**

A separately managed account is a type of investment account that allows investors to own individual securities directly. Unlike mutual funds or exchange-traded funds (ETFs), SMAs offer a customized investment portfolio tailored to an individual's specific needs and goals [1]. In an SMA, the investor owns the underlying securities, and the investment manager has discretion to buy and sell securities on behalf of the investor.

**How Does a Separately Managed Account Work?**

Here's a step-by-step overview